In [12]:
# Import libraries
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [13]:
print(torch.__version__)

2.3.0+cu118


In [14]:
import pandas as pd
df = pd.read_csv('train.csv')

In [15]:
Y = df['critical_temp'].values
X = df[[col for col in df.columns if col!="critical_temp"]].values

In [16]:
EPOCHS=100
# Create training and test sets
from sklearn.model_selection import train_test_split
#test_size is the percentage of the test size to the complete dataset
# random_state is the seed. A specific pseudorandom number to split the data set
# in order to produce same splitting every time we run the script.
features_train, features_test, target_train, target_test = train_test_split(X, Y, test_size = 0.25, random_state = 42)

In [18]:
#seed for Determinist behaviour
torch.manual_seed(0)
np.random.seed(0)

In [19]:
# Convert data to PyTorch tensors
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1,1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1,1)

In [20]:
# Define a neural network using `Sequential`
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(81, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64,32),
            torch.nn.ReLU(),
            torch.nn.Linear(32,1),
        )
    def forward(self, x):
        x = self.sequential(x)
        return x

In [21]:
# Initialize neural network
network = SimpleNeuralNet()
# Define loss function, optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(network.parameters(), lr=0.001)
# Define data loader
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)
# Compile the model using torch 2.0's optimizer
network = torch.compile(network)

In [22]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True
# Train neural network
for epoch in range(EPOCHS):
#This loop iterates over batches of data from the train_loader. 
# data contains the x_train, and target the y_train.    
    for batch_idx, (data, target) in enumerate(train_loader):
# clears possible old gradients from previous last step        
        optimizer.zero_grad()
# network processes input data and produces an output        
        output = network(data)
# computes the loss         
        loss = criterion(output, target)
# computes the gradient of the loss with respect to the network parameters        
        loss.backward()
# optimizer updates the model's parameters based on the gradients calculated in the previous step        
        optimizer.step()
    print("Epoch:", epoch+1, "\tLoss:", loss.item())

W0617 10:27:07.992000 12240 torch\_dynamo\convert_frame.py:824] WON'T CONVERT forward C:\Users\tdiak_000\AppData\Local\Temp\ipykernel_24152\3364497915.py line 12 
W0617 10:27:07.992000 12240 torch\_dynamo\convert_frame.py:824] due to: 
W0617 10:27:07.992000 12240 torch\_dynamo\convert_frame.py:824] Traceback (most recent call last):
W0617 10:27:07.992000 12240 torch\_dynamo\convert_frame.py:824]   File "C:\Users\tdiak_000\anaconda3\Lib\site-packages\torch\_dynamo\convert_frame.py", line 786, in _convert_frame
W0617 10:27:07.992000 12240 torch\_dynamo\convert_frame.py:824]     result = inner_convert(
W0617 10:27:07.992000 12240 torch\_dynamo\convert_frame.py:824]              ^^^^^^^^^^^^^^
W0617 10:27:07.992000 12240 torch\_dynamo\convert_frame.py:824]   File "C:\Users\tdiak_000\anaconda3\Lib\site-packages\torch\_dynamo\convert_frame.py", line 400, in _convert_frame_assert
W0617 10:27:07.992000 12240 torch\_dynamo\convert_frame.py:824]     return _compile(
W0617 10:27:07.992000 12240 t

Epoch: 1 	Loss: 458.98468017578125
Epoch: 2 	Loss: 415.4046630859375
Epoch: 3 	Loss: 365.6907043457031
Epoch: 4 	Loss: 268.4238586425781
Epoch: 5 	Loss: 262.84454345703125
Epoch: 6 	Loss: 219.2660369873047
Epoch: 7 	Loss: 363.6130676269531
Epoch: 8 	Loss: 384.7644958496094
Epoch: 9 	Loss: 354.20147705078125
Epoch: 10 	Loss: 255.7959747314453
Epoch: 11 	Loss: 431.1951599121094
Epoch: 12 	Loss: 201.426513671875
Epoch: 13 	Loss: 275.8376770019531
Epoch: 14 	Loss: 515.9777221679688
Epoch: 15 	Loss: 481.6201477050781
Epoch: 16 	Loss: 236.99356079101562
Epoch: 17 	Loss: 161.6465301513672
Epoch: 18 	Loss: 142.27479553222656
Epoch: 19 	Loss: 294.12091064453125
Epoch: 20 	Loss: 376.0154724121094
Epoch: 21 	Loss: 310.9591064453125
Epoch: 22 	Loss: 315.6485900878906
Epoch: 23 	Loss: 293.24658203125
Epoch: 24 	Loss: 208.0510711669922
Epoch: 25 	Loss: 235.03207397460938
Epoch: 26 	Loss: 126.41593170166016
Epoch: 27 	Loss: 231.76971435546875
Epoch: 28 	Loss: 228.00070190429688
Epoch: 29 	Loss: 295.0

In [23]:
# Evaluate neural network
with torch.no_grad():
    output = network(x_test)
    test_loss = float(criterion(output, y_test))
    print("Test MSE:", test_loss)

Test MSE: 213.3505859375


In [24]:
output_pandas = pd.DataFrame(output.numpy())

In [25]:
output_pandas

,0
0,9.253896
1,88.247826
2,53.364658
3,27.768547
4,5.686145
...,...
5311,19.858067
5312,1.436745
5313,24.342159
5314,68.837563
